In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\shyam\\OneDrive\\Documents\\GitHub\\AI-Bot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shyam\\OneDrive\\Documents\\GitHub\\AI-Bot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
#extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 37362


In [11]:
#text_chunks

In [12]:
#%pip uninstall sentence-transformers huggingface-hub -y
#%pip install sentence-transformers huggingface-hub --upgrade

In [13]:
#%pip install -U langchain-huggingface


In [14]:
from langchain_huggingface import HuggingFaceEmbeddings


In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
import os

def download_hugging_face_embeddings():
    local_model_path = "C:/Users/shyam/models/all-MiniLM-L6-V2"
    
    if not os.path.exists(local_model_path):
        raise FileNotFoundError(f"Model path does not exist: {local_model_path}")
    
    print(f"Loading model from: {local_model_path}")
    embeddings = HuggingFaceEmbeddings(model_name=local_model_path)
    print("Model loaded successfully.")
    
    return embeddings

embeddings = download_hugging_face_embeddings()


Loading model from: C:/Users/shyam/models/all-MiniLM-L6-V2


c:\Users\shyam\OneDrive\Documents\GitHub\AI-Bot\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded successfully.


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
#query_result

In [19]:
from dotenv import load_dotenv
load_dotenv()

False